Authors: Dror & Mariia

In [12]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

In [17]:
df = pd.read_csv('drive/My Drive/customer_segmentation.csv', encoding="ISO-8859-1") # from https://www.kaggle.com/carrie1/ecommerce-data
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### First, let's see what data do we have

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [23]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [24]:
df.duplicated().sum()

5268

### Issue N1: 25% of entries are null-valued customers. 

So, let's select this subgroup, save it separately and later check it. Should it be removed from our main data? 

Since we are clustering orders, these orders with undefined customers will be assigned to the same customer if we do some fillna - this is bad. Probably, we have to work with that data separately

In [55]:
null_customers = df[df.CustomerID.isna()]
null_customers.to_csv('drive/My Drive/null_customers.csv')  # later we will return to this
null_customers.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,dayofweek,weekofyear


Now, we will deal with data without NaNs:

In [29]:
# a few options
df1 = df.dropna(axis = 0, subset = ['CustomerID'])
df2 = df[~df.CustomerID.isna()] 
np.all(df1 == df2)

True

### Next, issue N2: duplicates

In [33]:
df1.duplicated().sum()

5225

In [34]:
df = df1.drop_duplicates()

In [41]:
df.info()  # maybe reset index?

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 37.6+ MB


In [42]:
dfr = df.reset_index()

### Further EDA

adding some new features, maybe it will be useful for clusterization

In [61]:
df['year'] = pd.DatetimeIndex(df['InvoiceDate']).year
df['month'] = pd.DatetimeIndex(df['InvoiceDate']).month
df['dayofweek'] = pd.DatetimeIndex(df['InvoiceDate']).dayofweek
df['weekofyear'] = pd.DatetimeIndex(df['InvoiceDate']).weekofyear
df['hour'] = pd.DatetimeIndex(df['InvoiceDate']).hour
df['dayofyear'] = pd.DatetimeIndex(df['InvoiceDate']).dayofyear

# looks like we got all information from the InvoiceDate, so
df = df.drop('InvoiceDate', axis=1)

In [62]:
# let's see how many unique values for each column maybe do we have
for col in df.columns:
    print(f'Column {col} has {len(np.unique(df[col]))} unique values')

Column InvoiceNo has 22190 unique values
Column StockCode has 3684 unique values
Column Description has 3896 unique values
Column Quantity has 436 unique values
Column UnitPrice has 620 unique values
Column CustomerID has 4372 unique values
Column Country has 37 unique values
Column year has 2 unique values
Column month has 12 unique values
Column dayofweek has 6 unique values
Column weekofyear has 51 unique values
Column hour has 15 unique values
Column dayofyear has 298 unique values


Regarding 6 days in a week :) probably, the orders are just not processed on some day of the week and hence moved to the next day. we will see it further

Okay, possible options:

    - plot boxplots for features with small number of unique values

    - for Country, week, dayofweek, dayofyear and month we can aggregate total number of orders, total sum of orders made, etc


In [54]:
# plt.figure(figsize=(15, 25))
# sns.boxplot(x="UnitPrice", y="Country", data=df)

In [63]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,year,month,dayofweek,weekofyear,hour,dayofyear
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010,12,2,48,8,335
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010,12,2,48,8,335
